In [2]:
import numpy as np
import tensorflow as tf
#from tensorflow.keras.layers import TextVectorization
import pickle

2025-06-02 11:07:47.367290: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748862467.570465      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748862467.634741      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def load_from_disk(path):

    with open(path, "rb") as f:
        loaded_list = pickle.load(f)

    return loaded_list

In [4]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [5]:
loaded_img_paths=load_from_disk('/kaggle/input/image-paths/train_image_paths.pkl')

In [6]:
def load_and_preprocess_image(path):
    path = tf.strings.join(['/kaggle/input/flickr8k/Images/', path])
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize_with_pad(img, 224, 224)
    img = tf.cast(img, tf.float32)
    img = preprocess_input(img)  # Normalize for ResNet-50
    return img

In [7]:
data_encoder_set=tf.data.Dataset.from_tensor_slices(loaded_img_paths) \
.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE) \
.batch(16)

I0000 00:00:1748862482.868048      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [8]:
# strategy = tf.distribute.MirroredStrategy()
# with strategy.scope():
encoder=ResNet50(include_top=False,
    weights='imagenet',
    input_shape=(224,224,3))
    

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
import numpy as np

results = []

for batch in data_encoder_set:
    # Forward pass
    pred = encoder(batch, training=False)  # use model() instead of .predict()
    results.append(pred.numpy())           # move to CPU as NumPy

# Concatenate all results on CPU
output_result = np.concatenate(results, axis=0)

print("Final output shape:", output_result.shape)


I0000 00:00:1748862486.389205      19 cuda_dnn.cc:529] Loaded cuDNN version 90300


Final output shape: (28320, 7, 7, 2048)


In [10]:
output_result[1]

array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.75875163,  0.        , ...,  0.        ,
          0.        ,  2.087933  ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  

In [11]:
np.save('/kaggle/working/processed_train_images.npy',output_result)
print("Saved")

Saved


In [12]:
# output_result=encoder.predict(data_encoder_set)